# Detect static red cells using Laplacian of Gaussian(LoG) filter and intensity threshold

* Put this jupyter notebook, the helper_functions file and a red channel recording in the same folder.

* Currently supporting tiff or isxd format. If the input file is a tiff, the user also needs to provide an isxd file with the same frame size (for metadata transfer, required for multi-color registration).

* Adjust the [kernel size](#kernel_size) for the LoG filter, the image intensity threshold [#1](#intensity_threshold_1) and [#2](#intensity_threshold_2), the cell size threshold [#1](#cell_size1) and [#2](#cell_size2) until optimal detection has been achieved.

* Require manual installation of cv2 package before opening jupyter notebook. [reference](https://anaconda.org/conda-forge/opencv)
`conda install -c conda-forge opencv`

## 0. Prepare libraries and load a static red recording for processing

### 0.1 import libraries and helper functions


In [ ]:
import isx
import numpy as np
import cv2
from pathlib import Path  # to work with dir
import os
from helper_functions import *

### 0.2 load a tiff image or a short recording in isxd format. The isxd file will be processed and converted to tiff format.

#### [isxd input only] spatial filter the isxd recording, project a mean image and convert to Tiff format

In [ ]:
# change the input file name base on the file type: tiff or isxd
# leave the tiff filename as default if the input file is an isxd file
# a separate isxd file is required to copy metadata if analyzing a Tiff file

tiff_filename = 'default.tiff'
isxd_filename = '2022-05-30-18-54-51_video_red-PP.isxd'

if tiff_filename == 'default.tiff':
    input_isxd = Path(os.getcwd(), isxd_filename)
    projected_red = project_isxd(input_isxd)
    output_tiff = Path(os.getcwd(), input_isxd.stem + '.tiff')
    isx.export_isxd_image_to_tiff(projected_red, str(output_tiff))
    metadata_isxd = input_isxd
else:
    output_tiff = Path(os.getcwd(), tiff_filename)
    metadata_isxd = Path("path/to/file")

### 0.3 display the unfiltered red channel image

In [ ]:
img_tiff = cv2.imread(str(output_tiff), -1)  # gray scale conversion

# zero meaned image is critical for LoG filter
img_demean = img_tiff - np.mean(img_tiff)

title = 'showing unfiltered mean projection image'
show_img(img_demean, title=title, percentile=99)

## 1. Enhance red cell contrast by convolving the image with a Laplacian of Gaussian (LoG) filter

###  1.1 convolve the image with a LoG filter and display the convolved image

<a name="kernel_size"> adjust the kernel_size</a>

In [ ]:
# the average cell diameter in pixels measured with a line tool in the IDPS
cell_diameter = 7

#####################################################################
# Adjust the kernel size/n to enhance the contrast of the cell body #
n = 2  # default 2
kernel_size = cell_diameter * n
#####################################################################

img_LoG = LoG_convolve(img_demean, kernel_size=kernel_size)

title = 'the image after convolution with a LoG filter'
show_img(img_LoG, title=title)

### 1.2 apply intensity threshold to review suprathreshold ROIs

<a name="intensity_threshold_1"> adjust the image intensity threshold 1</a>

In [ ]:
################################################################
# adjust the n_std to detect pixels with luminance
# n times of standard deviation above the mean image intensity
n_std = 1
################################################################

LoG_footprints = threshold_img(img_LoG, std_factor=n_std)

title = f'detecting footprints that are {n_std} standard deviation above ' \
    'the mean image intensity'
show_img(img_LoG, footprints=LoG_footprints, title=title)

### 1.3 reject dark background that was enhanced with a second intensity threshold

<a name="intensity_threshold_2"> adjust the image intensity threshold 2 to reject low luminance background</a>

In [ ]:
########################################################################
# adjust the image_threshold2 to reject background such as blood vessels
# that could also be enhanced during LoG filtering
image_threshold2 = 0.005
########################################################################

LoG_footprints = shrink_footprints(img_demean,
                                   LoG_footprints,
                                   threshold=image_threshold2)

title = "LoG filtered ROIs that are " \
    f"brighter than {image_threshold2*100}% of the raw image maximal luminance"

show_img(img_demean, LoG_footprints, title=title)

## 2. Segment footprints with different sizes

### 2.1 find non overlapping footprints by choosing a small size range

<a name="cell_size1"> adjust the cell size threshold set 1</a>

In [ ]:
##############################################################
# adjust the size multipliers to filter cells with proper size
max_size_multiplier = 3
min_size_multiplier = 0.2
##############################################################

average_cell_area = np.pi * (cell_diameter * cell_diameter) / 4
min_size = int(average_cell_area * min_size_multiplier)
max_size = int(average_cell_area * max_size_multiplier)

small_footprints = footprint_filter(img_LoG, LoG_footprints, max_size,
                                    min_size)

title = "overlaying small size footprints"
show_img(img_demean, small_footprints, title=title)

### 2.2 find slightly overlapping footprints by choosing a medium size range

<a name="cell_size2"> adjust the cell size threshold set 2</a>

In [ ]:
#############################################################
# adjust the size multiplier to detect overlapping footprints
max_size_multiplier2 = 8
min_size_multiplier2 = 3
#############################################################

min_size = int(average_cell_area * min_size_multiplier2)
max_size = int(average_cell_area * max_size_multiplier2)

medium_footprints = footprint_filter(img_demean, LoG_footprints, max_size,
                                     min_size)
title = "showing medium sized footprints"
show_img(img_demean, medium_footprints, title=title)

### 2.3 split medium sized ROIs with watershed filter

In [ ]:
# split multiple-cell ROIs with watershed filter
splitted_footprints, _ = split_merge_cells(img_demean,
                                           LoG_footprints,
                                           medium_footprints,
                                           cell_size=cell_diameter * 1.5)

title = 'showing the splitted footprints from the two categories'
show_img(img_demean, splitted_footprints, title=title, plot_individual=True)


### 2.4 combine the two categories

In [ ]:
final_footprints = merge_cells(small_footprints, splitted_footprints)
title = 'showing the final footprints from the two categories'
show_img(img_demean, final_footprints, title=title, plot_individual=True)

## 3. Export footprints to an isxd file. The cellsets can be imported into IDPS for multicolor registration.

In [ ]:
# new file will be generated with a "autoseg" suffix,
# which can be imported into Inscopix processing software
# to further inspect individual red cell or
# perform multicolor registration.

footprints_export_to_isxd(metadata_isxd, final_footprints, suffix="_autoseg")